## **Load libraries and data**

In [13]:
#!pip install --upgrade pythainlp
!python -m pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable


In [14]:
import pandas as pd
import pythainlp
import gensim
#import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [15]:
df = pd.read_csv('https://github.com/Surawich2021/Surawich-6310422083/raw/main/Homework11%20-%20Voice%20of%20Customer/Wongnai%20Reviews%20-%20Small.csv')

In [16]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize Words**

In [17]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [18]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [19]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [20]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [21]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [22]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [23]:
num_topics = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 8.89 s


In [24]:
#pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary, mds='mmds') 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.138768 -0.084722       1        1  11.109502
4      0.003831 -0.164270       2        1  10.925169
16    -0.213302 -0.076614       3        1   7.442588
7      0.075388 -0.169297       4        1   6.914840
19     0.145655 -0.210258       5        1   6.400069
29     0.214280 -0.116744       6        1   5.052926
25     0.123032 -0.110257       7        1   4.648831
24     0.067225 -0.288835       8        1   4.546604
21    -0.162235 -0.254248       9        1   4.457858
15    -0.088353 -0.151179      10        1   3.867284
9     -0.163415  0.042805      11        1   3.677330
10    -0.229023  0.009753      12        1   3.556014
28    -0.261373 -0.142584      13        1   2.889057
8     -0.047781 -0.228326      14        1   2.721805
3     -0.120684  0.246024      15        1   2.384879
27    -0.167915  0.163420      16        1   2.258584
14    -0.220939  0.126613      17        1   2.149171
17    -0.072901  0.036107      18        1   2.120708
23     0.003786 -0.051268      19        1   2.066721
26     0.314881 -0.002398      20        1   1.841817
13     0.226589  0.030423      21        1   1.659891
2      0.106388  0.271899      22        1   1.391068
20    -0.040596  0.225731      23        1   1.154196
6      0.053126  0.044155      24        1   1.043407
5     -0.014343  0.137300      25        1   0.991944
22     0.044338  0.206140      26        1   0.887523
18     0.100868  0.060837      27        1   0.612266
11     0.158912  0.112813      28        1   0.508185
12     0.113570  0.155094      29        1   0.391341
0      0.189758  0.181885      30        1   0.328424, topic_info=          Term        Freq       Total Category  logprob  loglift
36          ชา  237.000000  237.000000  Default  30.0000  30.0000
914     ไข่มุก   98.000000   98.000000  Default  29.0000  29.0000
438         นม  139.000000  139.000000  Default  28.0000  28.0000
2         กาแฟ  274.000000  274.000000  Default  27.0000  27.0000
40        นั่ง  174.000000  174.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
41      น่ารัก    0.386604   55.112395  Topic30  -5.7010   0.7589
860   เบเกอรี่    0.386585   19.303985  Topic30  -5.7010   1.8079
596        เบา    0.386565   18.071303  Topic30  -5.7011   1.8738
256     ตกแต่ง    0.386539   55.425590  Topic30  -5.7011   0.7531
2576       เป้    0.386529    2.685795  Topic30  -5.7012   3.7801

[2188 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
4507     23  0.691980                                                   
3691     12  0.786228                                                   
3849     23  0.691973                                                   
318       1  0.130350                                                  "
318       2  0.260700                                                  "
...     ...       ...                                                ...
4651     12  0.710193                                                  •
4693     18  0.852798                                                 ••
3148     25  0.494155  …................................................
2129      2  0.747716                                                  ′
2130      2  0.747716                                                  ″

[5690 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 17, 8, 20, 30, 26, 25, 22, 16, 10, 11, 29, 9, 4, 28, 15, 18, 24, 27, 14, 3, 21, 7, 6, 23, 19, 12, 13, 1])

In [25]:
model.show_topic(1)

[('อาหาร', 0.0157662),
 ('เมนู', 0.01339695),
 ('อร่อย', 0.012515231),
 ('ทาน', 0.011800207),
 ('รสชาติ', 0.009444948),
 ('ที่นี่', 0.008708763),
 ('ผม', 0.00819092),
 ('ดู', 0.0080177635),
 ('กาแฟ', 0.007389711),
 ('คน', 0.0069788327)]

In [26]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [27]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",7,0.154460
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",10,0.778061
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",2,0.989049
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",15,0.996296
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...",28,0.994604
